In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import newton

In [8]:
def GeneratePathsGBM(NoOfPaths, NoOfSteps, t1, T1, T, sigma, lambda1, frate, corp, K, numeraire):
    #finding the 'dt' value the limits of integration are already fixed i.e., from s to T i.e., time to maturity
    dt = t1 / float(NoOfSteps)
    if T/dt != int(T/dt) or t1/dt != int(t1/dt) or T1/dt != int(T1/dt):
      print("enter NoofSteps such that T and t1 are exact multiples of dt")
    time = np.arange(0,T+dt,dt)
    global NoOfSteps1
    NoOfSteps1 = NoOfSteps
    #generating standard normal distribution values for  'dW'
    Z = np.random.normal(0.0, 1.0, [NoOfPaths,len(time)])
    #creating vector for storing the interest rate values
    r = np.zeros([NoOfPaths, len(time) ])

    r[:, 0] = frate



    for i in range(0, len(time)-1):
      # Making sure that samples from a normal have mean 0 and variance 1
      if NoOfPaths > 1:
          Z[:, i] = (Z[:, i] - np.mean(Z[:, i])) / np.std(Z[:, i])

      #finding the value of nut for each time interval
      nut = 0 + lambda1 * frate + (np.power(sigma,2) / (2 * lambda1)) * (1 - np.exp(-2 * lambda1 * time[i]))

      #finding value of interest rate at every time step
      if numeraire != "tforward":
        r[:,i+1] = r[:,i] + (nut - lambda1 * r[:,i]) * dt + sigma * np.power(dt,0.5) * Z[:, i]
      else:
        r[:,i+1] = r[:,i] + (nut - lambda1 * r[:,i] - np.power(sigma,2) * fb(time[i],T)) * dt + sigma * np.power(dt,0.5) * Z[:, i]

    #defining functions for the exact bond price formula
    def fb(t,T):
      return (1/lambda1) * (1 - np.exp(-lambda1*(T - t)))

    def fa(t,T):
      return (np.exp(-1*frate*T)/np.exp(-1*frate*t)) * (np.exp(fb(t,T) * (frate) - (np.power(sigma,2)/(4 * lambda1)) * (1 - np.exp(-2 * lambda1 * t)) * np.power(fb(t,T),2)))

    def P(t,T,rt):
      return (fa(t,T) * np.exp(-1 * fb(t,T) * rt))

    value=np.zeros([NoOfPaths,len(time)-NoOfSteps])

    for i in range(NoOfSteps,len(time)):
      value[:,i-NoOfSteps] = fa(time[i],T)*np.exp(-fb(time[i],T)*r[:,i])

    disv = value[:,0] * np.exp(-1 * np.sum(r[:,0:NoOfSteps] * dt))
    #finding the value of the option at T1
    opt = np.maximum(value[:,int(T1/dt - NoOfSteps)] - K,0)
    #discounting the option value to time "t1" over simulated paths of interest rates
    disopt = opt * np.exp(-1 * np.sum(r[:,NoOfSteps:int(T1/dt)]*dt,axis=1))
    #defining the zero bond coupon exact formula
    def zbc(t,T1,T,K, rt):
      sigmap = sigma * np.power(((1 - np.exp(-2* lambda1 * (T1 - t)))/(2*lambda1)),0.5) * fb(T1,T)
      h = (1/sigmap)*(np.log(P(t,T,rt)/(P(t,T1,rt)*K))) + (sigmap/2)
      if sigmap==0:
        h=float('inf')
      return P(t, T, rt) * norm.cdf(h) - K * P(t,T1, rt) * norm.cdf(h - sigmap)

    #Jamshidian decomposition to convert a max over summation to summation of maximums

    #summation function to find r* using newton-raphson numerical method
    def sumbonds(r_star):
      sum = 0
      for i in range(T1+1,T):
        sum += K * 1 * P(T1, i, r_star)
      sum += (1 + K * 1) * P(T1, T, r_star)
      return sum - 1

    #array of dates where the exchanges take place
    t_array = np.arange(T1 + 1, T + 1 , 1)
    #finding r_star which causes summation of discounted cashflows to be equal to the strik.
    r_star = newton(sumbonds, K, maxiter=10000)
    #Finding the value of zero coupon bonds with r* as the interest rate to be passed into the zero coupon bond pricing function
    x_array = np.transpose(np.array([P(T1, t_array[i], r_star) for i in range(len(t_array))]))
    #price of the swaption using zero bond coupon formula after jamshidian decomposition
    swaptionvalue = np.sum((np.array([K* 1 * zbc(t1, T1, t_array[i], x_array[i], frate) if i!=len(t_array)-1 else (K+1) * zbc(t1, T1, t_array[-1], x_array[-1],frate) for i in range(len(t_array))])))
    #price of the swaption using maximum of summation of (strike - discounted cashflows)
    swaptionvalueform = np.exp(-1 * np.sum(r[:,NoOfSteps:int(T1/dt)]*dt,axis=1)) * (np.maximum(np.sum(np.transpose(np.array([P(T1,t_array[i],frate) * 1 * (-1 * (((P(T1, [t_array[i-1] if i !=0 else T1][0], frate)/P(T1, t_array[i], frate)) - 1)/1) + K) for i in range(len(t_array))]))),0))
    #swaptionvalueform = np.exp(-1 * np.sum(r[:,NoOfSteps:int(T1/dt)]*dt,axis=1)) * (np.maximum(np.sum(np.transpose(np.array([P(T1,t_array[i],r[:,int(t_array[i]/dt)]) * 1 * (-1 * (((P(T1, [t_array[i-1] if i !=0 else T1][0], r[:,int(t_array[i]/dt)])/P(T1, t_array[i], r[:,int(t_array[i]/dt)])) - 1)/1) + K) for i in range(len(t_array))])),axis = 1),0))
    return {"swaptionvalueform":swaptionvalueform,"swaptionvalue":swaptionvalue,"disv":disv,"value":value,"dt":dt,"time":time,"opt":opt,"disopt":disopt}

In [9]:
k=GeneratePathsGBM(10, 3000, 3, 5, 10, 0.01, 1, 0.1, "c", 0.5, "moneybank")

In [10]:
#this is the array containing the values of swaption at time 't' found from the closed form formula
k["swaptionvalue"]

1.2093535417021122

In [11]:
#this is the array containing the values of swaption at time 't' found from undecomposed swaption formula then discounted over the interest rate paths simulated
k["swaptionvalueform"]

array([1.20230885, 1.21767674, 1.20836162, 1.19941469, 1.21234189,
       1.2263507 , 1.21707112, 1.20604184, 1.21111766, 1.19198267])